In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Load the disaster data
disaster_df = pd.read_csv(r'D:\OneDrive\StudyMat\CarbonSci-cle-Data\data\raw\disaster\fema_disaster_with_noaa_monthly_temp_combined.csv')

C:\Users\Alex Hoang\AppData\Local\Temp\ipykernel_37516\4108214428.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  disaster_df = pd.read_csv(r'D:\OneDrive\StudyMat\CarbonSci-cle-Data\data\raw\disaster\fema_disaster_with_noaa_monthly_temp_combined.csv')


In [3]:
# remove the rows with missing values
disaster_df = disaster_df.drop(columns=['Unnamed: 0', 'last_ia_filing_date'])
disaster_df = disaster_df.dropna()


In [4]:
# Count total number of disaster events of each state per year based on the number of time the state appears that year
# Ensure relevant columns are of the correct data type
disaster_df['State'] = disaster_df['State'].astype(str)
disaster_df['year'] = disaster_df['year'].astype(str)

# Group by 'State' and 'year' and count the occurrences
disaster_counts = disaster_df.groupby(['State', 'year']).size().reset_index(name='total_disasters')

# Pivot the DataFrame to get the desired format
pivot_df = disaster_counts.pivot(index='year', columns='State', values='total_disasters').fillna(0).astype(int)


In [ ]:
# Plot the total co2 emissions per year of each state and plot the total number of disaster events per year of each state corresponding to the co2 emissions
def plot_save(disaster_df, co2_dir):
    for file in os.listdir(co2_dir):
        if file.endswith('.csv'):
            co2_df = pd.read_csv(os.path.join(co2_dir, file))
            co2_df = co2_df.dropna()
            state = file.split('-')[0]
            
            co2_df['State'] = state
            co2_df['datetime'] = pd.to_datetime(co2_df['datetime'])
            co2_df['year'] = co2_df['datetime'].dt.year.astype(str)

            co2_df = co2_df.groupby(['State', 'year'])['co2_emissions'].sum().reset_index(name='total_co2_emissions')
            co2_pivot_df = co2_df.pivot(index='year', columns='State', values='total_co2_emissions').fillna(0).astype(int)
            fig, ax = plt.subplots(2, 1, figsize=(15, 10))
            sns.heatmap(co2_pivot_df, ax=ax[0], cmap='coolwarm', cbar_kws={'label': 'Total CO2 Emissions'})
            sns.heatmap(pivot_df, ax=ax[1], cmap='coolwarm', cbar_kws={'label': 'Total Disasters'})
            ax[0].set_title('Total CO2 Emissions per year of each state')
            ax[1].set_title('Total number of disaster events per year of each state')
            fig.suptitle('CO2 Emissions vs Disasters')
            plt.tight_layout()
            plt.savefig(f'{file[:-4]}_co2_vs_disasters.png')
            plt.show()